# Tarea 2 – ISIS 4221, Notebook I

## Paso 0: Importación de Librerías Necesarias

In [ ]:
import os
import json
from glob import glob
from itertools import chain
from re import compile, DOTALL, findall
from collections import Counter, defaultdict

import nltk
from nltk.tokenize import sent_tokenize
from pandas import DataFrame, Series
from tqdm import tqdm

nltk.download('punkt')

: 

In [2]:
# Define los directorios que se deben crear
ngram_models_dir = './data/ngram_models/'
train_test_dir = './data/train_test/'

# Crea los directorios si no existen
os.makedirs(ngram_models_dir, exist_ok=True)
os.makedirs(train_test_dir, exist_ok=True)

## Paso 1: Definición de las Funciones Necesarias

In [3]:
def read_file(f: str) -> str:
    """
    Lee el contenido de un archivo y maneja posibles errores de codificación.

    Args:
        f (str): La ruta al archivo que se desea leer.

    Returns:
        str: El contenido del archivo como una cadena de texto.

    Notas:
        - La función intenta leer el archivo con codificación 'utf-8' por defecto.
        - Si ocurre un error de decodificación (UnicodeDecodeError), se reintenta con la codificación 'latin1'.
        - Este enfoque es útil cuando se manejan archivos de texto con codificaciones mixtas o desconocidas.
    """
    try:
        with open(f, 'r', encoding='utf-8') as file:
            txt = file.read()
    except UnicodeDecodeError:
        with open(f, 'r', encoding='latin1') as file:
            txt = file.read()
    
    return txt

def clean_text(txt: str) -> str:
    """
    Limpia el texto eliminando correos electrónicos, etiquetas HTML, caracteres no alfanuméricos,
    y otros elementos no deseados.

    Args:
        txt (str): El texto que se desea limpiar.

    Returns:
        str: El texto limpio y normalizado.

    Pasos:
        - Convertir el texto a minúsculas para normalizarlo.
        - Eliminar correos electrónicos, etiquetas HTML, y ciertas palabras clave usando expresiones regulares.
        - Reemplazar secuencias de puntos múltiples con un solo espacio.
        - Eliminar repeticiones de caracteres consecutivos (más de dos veces) que no sean necesarias.
        - Reemplazar guiones por espacios para evitar palabras concatenadas por guiones.
        - Sustituir signos de puntuación específicos (?, !, :) por un punto.
        - Eliminar caracteres no alfanuméricos, excepto puntos, espacios y saltos de línea.
        - Sustituir números por la palabra 'NUM' para normalizar las secuencias numéricas.
        - Reemplazar múltiples saltos de línea consecutivos con un solo punto.
        - Convertir saltos de línea individuales en espacios.
        - Reemplazar múltiples espacios consecutivos por un solo espacio.

    Notas:
        - El uso de expresiones regulares precompiladas mejora la eficiencia al aplicar la limpieza a múltiples textos.
    """

    # Compila las expresiones regulares fuera de la función para mejorar la eficiencia
    email_re = compile(r'\S*@\S*\s?|from: |re: |subject: |urllink|maxaxaxaxaxaxaxaxaxaxaxaxaxaxax')
    punctuation_re = compile(r'[?!:]')
    non_alphanumeric_re = compile(r'[^A-Za-z0-9. \n]')
    numbers_re = compile(r'\b\d{1,3}(,\d{3})*(\.\d+)?\b')
    multiple_newlines_re = compile(r'\n{2,}')
    single_newline_re = compile(r'\n')
    multiple_dots_re = compile(r'\.\.+')
    multiple_spaces_re = compile(r'\s+')
    multiple_char_repetition_re = compile(r'(.)\1{2,}')
    
    # Convierte el texto a minúsculas
    txt = txt.lower()

    # Elimina correos electrónicos, etiquetas dentro de <>, y ciertas palabras clave
    txt = email_re.sub('', txt)

    # Reemplaza secuencias de puntos múltiples con un solo espacio
    txt = multiple_dots_re.sub(' ', txt)

    # Elimina repeticiones de caracteres consecutivos (más de dos veces)
    txt = multiple_char_repetition_re.sub(r'\1', txt)

    # Reemplaza guiones por espacios
    txt = txt.replace('-', ' ')

    # Sustituye signos de puntuación específicos por un punto
    txt = punctuation_re.sub('.', txt)

    # Elimina caracteres no alfanuméricos excepto puntos, espacios y saltos de línea
    txt = non_alphanumeric_re.sub('', txt)

    # Sustituye números por 'NUM'
    txt = numbers_re.sub('NUM', txt)

    # Reemplaza múltiples saltos de línea consecutivos por un solo punto
    txt = multiple_newlines_re.sub('.', txt)

    # Convierte saltos de línea individuales en espacios
    txt = single_newline_re.sub(' ', txt)

    # Reemplaza múltiples espacios consecutivos por un solo espacio
    txt = multiple_spaces_re.sub(' ', txt)

    # Elimina espacios en blanco al inicio y final del texto
    return txt.strip()

def extract_and_process_text_from_xml(file_path: str) -> list:
    """
    Extrae y procesa el texto contenido entre etiquetas <post> en un archivo XML.

    Args:
        file_path (str): La ruta al archivo XML del cual se extraerá el texto.

    Returns:
        list: Una lista de diccionarios, donde cada diccionario representa una oración extraída y procesada
              con las claves:
              - 'text': La oración procesada.
              - 'length': La longitud de la oración en número de palabras.

    Notas:
        - La función primero lee el archivo XML utilizando `read_file`.
        - Luego extrae el contenido de todas las etiquetas <post> usando expresiones regulares.
        - Cada fragmento de texto extraído se limpia utilizando la función `clean_text`.
        - El texto limpio se divide en oraciones, que se formatean y almacenan en una lista de diccionarios.
        - Cada oración es rodeada por etiquetas <s> y </s> para indicar su inicio y fin.
        - Las oraciones que contienen una sola palabra o están vacías se descartan.
    """
    # Lee el contenido del archivo XML
    xml_content = read_file(file_path)
    
    # Extrae el contenido entre las etiquetas <post> y </post> utilizando expresiones regulares
    post_matches = findall(r'<post>(.*?)</post>', xml_content, DOTALL)
    
    df_rows = []
    
    for post in post_matches:
        # Limpia el texto extraído utilizando la función clean_text
        cleaned_post = clean_text(post.strip())
        
        # Divide el texto limpio en oraciones
        sentences = [f'<s> {s.strip()} </s>' for s in sent_tokenize(cleaned_post) if len(s.strip().split()) > 1]
        
        # Crea un diccionario para cada oración con su texto y longitud
        df_rows.extend([{
            'text': s,
            'source': file_path,
            'length': len(s.split())
        } for s in sentences])

    return df_rows

def create_ngrams(sentence: str, n: int, unique_tokens: dict = None) -> list:
    """
    Genera n-gramas a partir de una oración dada.

    Args:
        sentence (str): La oración de entrada de la cual se generarán los n-gramas.
        n (int): La longitud de los n-gramas a generar.
        unique_tokens (dict, opcional): Un diccionario que mapea ciertos tokens a un valor único,
            típicamente utilizado para reemplazar tokens poco frecuentes por un marcador como '<UNK>'.
            Por defecto es None.

    Returns:
        list: Una lista de n-gramas, donde cada n-grama se representa como una tupla de palabras.

    Notas:
        - Si se proporciona `unique_tokens`, cualquier palabra en la oración que aparezca en 
          `unique_tokens` será reemplazada según el mapeo en `unique_tokens`.
        - La función devuelve n-gramas en forma de un generador para ahorrar memoria, 
          especialmente útil al procesar grandes corpus.
    """
    words = sentence.split()  # Divide la oración en palabras
    
    if unique_tokens:
        # Genera n-gramas usando un generador para reemplazar las palabras según el diccionario unique_tokens
        return (tuple([unique_tokens.get((w,), w) for w in words[i:i+n]]) 
                for i in range(len(words) - n + 1))
    
    # Genera los n-gramas usando un generador sin realizar reemplazos
    return (tuple(words[i:i+n]) for i in range(len(words) - n + 1) if len(words[i:i+n]) == n)

def create_ngram_model(n_gram: int, text_corpus: Series)-> tuple[Counter[int], defaultdict[int]]:
    """
    Crea un modelo de n-gramas a partir de un corpus de texto.

    Args:
        n_gram (int): El tamaño de los n-gramas a generar.
        text_corpus (pd.Series): Una serie de Pandas que contiene el corpus de texto, 
                                 donde cada entrada es una oración o texto.

    Returns:
        tuple: Dos elementos:
            - ngram_counts (Counter): Un contador que almacena las frecuencias de los n-gramas en el corpus.
            - final_unigram (defaultdict): Un diccionario con los (n-1)-gramas más frecuentes y su conteo, 
                                           con un marcador especial `<UNK>` para los menos frecuentes.

    Notas:
        - La función primero cuenta los (n-1)-gramas para identificar los tokens únicos que serán 
          reemplazados por `<UNK>`.
        - Luego, se cuentan los n-gramas completos utilizando los reemplazos identificados.
    """    

    # Cuenta las frecuencias de (n-1)-gramas en el corpus de texto
    unigram_counts = Counter(chain.from_iterable(
        text_corpus.apply(lambda x: create_ngrams(x, n=n_gram-1, unique_tokens=None))
    ))

    final_unigram = defaultdict(int)
    unique_tokens = defaultdict(int)

    # Identifica tokens únicos y construir el diccionario final de 'unigrama'
    for ngram, count in unigram_counts.items():
        if '' in ngram:
            continue
        if count < 2:
            unique_tokens[ngram] = ('<UNK>',) * (n_gram-1)
        else:
            final_unigram[ngram] = count

    final_unigram[('<UNK>',)*(n_gram-1)] = len(unique_tokens)

    # Cuenta las frecuencias de los n-gramas completos usando los reemplazos identificados
    ngram_counts = Counter(chain.from_iterable(
        text_corpus.apply(lambda x: create_ngrams(x, n=n_gram, unique_tokens=unique_tokens))
    ))

    return ngram_counts, final_unigram

def save_ngram_model(ngram_counts, final_unigram, model_name):
    """
    Guarda el modelo de n-gramas en un archivo JSON.

    Args:
        ngram_counts (Counter): Los n-gramas y sus conteos.
        model_name (str): Nombre del archivo donde se guardará el modelo.
    """
    with open(f'./data/ngram_models/{model_name}.json', 'w') as f:
        json.dump({ 
            'ngram_counts': { ' '.join(str(w) for w in k): v for k, v in ngram_counts.items() },
            'final_unigram': { ' '.join(str(w) for w in k): v for k, v in final_unigram.items() }
            }, f)

    print(f"{model_name} saved.")

## Paso 2: Procesamiento y Almacenamiento de 20 Newsgroups (20N)

In [5]:
# Obtiene una lista de archivos en el directorio './raw_data/20news-18828/' que contiene subdirectorios para cada newsgroup
# NOTA: Actualizar segun donde se ubique el dataset
files_20n = glob('./raw_data/20news-18828/*/*')

# Cuenta el número total de archivos encontrados
print(f"Número total de archivos encontrados: {len(files_20n)}")

Número total de archivos encontrados: 18828


In [6]:
# Inicializa una lista para almacenar las filas procesadas
df_news_rows = []

# Procesa cada archivo encontrado
for f in tqdm(files_20n):
    # Extrae la categoría (nombre del subdirectorio) a partir de la ruta del archivo
    category = os.path.basename(os.path.dirname(f))
    
    # Lee el contenido del archivo
    txt = read_file(f)
    
    # Limpia el texto leído
    txt_cln = clean_text(txt)
    
    # Divide el texto limpio en oraciones, añadiendo etiquetas de inicio y fin de oración
    sentences = [f'<s> {s.strip()} </s>' for s in sent_tokenize(txt_cln) if len(s.strip().split()) > 1]
    
    # Crea un diccionario para cada oración con el texto, la fuente del archivo, la categoría y la longitud de la oración
    df_news_rows.extend([{
        'text': s,
        'source': f,
        'category': category,  # Añade la categoría como una columna adicional
        'length': len(s.split())
    } for s in sentences])

# Crea un DataFrame a partir de las filas procesadas
df_news = DataFrame(df_news_rows)

# Guarda el DataFrame en un archivo Parquet para su almacenamiento eficiente
df_news.to_parquet('./data/20news.parquet', index=False)

df_news.head(10)

100%|██████████| 18828/18828 [04:24<00:00, 71.23it/s]


,text,source,category,length
0,<s> mathew alt.atheism faq. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,5
1,<s> atheist resources.archive name. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,5
2,<s> atheismresources alt atheism archive name....,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,7
3,<s> resources last modified. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,5
4,<s> NUM december 1992 version. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,6
5,<s> atheist resources. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,4
6,<s> addresses of atheist organizations. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,6
7,<s> usa.freedom from religion foundation.darwi...,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,26
8,<s> ffrf p.o. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,4
9,<s> box NUM madison wi 53701. telephone. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,8


In [7]:
# Divide aleatoriamente el DataFrame en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
df_news_train = df_news.sample(frac=0.8, random_state=42)
df_news_test = df_news.drop(df_news_train.index)

# Guarda los conjuntos de entrenamiento y prueba en archivos Parquet separados
df_news_train.to_parquet('./data/train_test/20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_training.parquet', index=False)
df_news_test.to_parquet('./data/train_test/20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_testing.parquet', index=False)

In [8]:
# Genera los modelos de N-gramas
unigram_counts_20n, unigram_20n = create_ngram_model(1, df_news['text'])
bigram_counts_20n, bigram_20n = create_ngram_model(2, df_news['text'])
trigram_counts_20n, trigram_20n = create_ngram_model(3, df_news['text'])

In [9]:
# Guarda los modelos de N-gramas
save_ngram_model(unigram_counts_20n, unigram_20n, '20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_unigrams')
save_ngram_model(bigram_counts_20n, bigram_20n, '20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_bigrams')
save_ngram_model(trigram_counts_20n, trigram_20n, '20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_trigrams')

20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_unigrams saved.
20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_bigrams saved.
20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_trigrams saved.


## Paso 3: Procesamiento y Almacenamiento de Blog Authorship Corpus (BAC)

In [10]:
# Obtiene una lista de archivos XML en el directorio './raw_data/blogs/'
# Nota: Actualizar ruta segun corresponda
files_bac = glob('./raw_data/blogs/*')

# Cuenta el número total de archivos encontrados
len(files_bac)

19320

In [11]:
# Inicializa una lista para almacenar las filas procesadas
df_bac_rows = []

# Procesa cada archivo XML encontrado
for f in tqdm(files_bac):
    # Extraer y procesar el texto del archivo XML, y agregar las filas al DataFrame
    df_bac_rows.extend(extract_and_process_text_from_xml(f))

# Crea un DataFrame a partir de las filas procesadas
df_bac = DataFrame(df_bac_rows)

# Muestra el DataFrame resultante
print(df_bac.head())

# Guarda el DataFrame en un archivo Parquet para su almacenamiento eficiente
df_bac.to_parquet('./data/bac.parquet', index=False)

100%|██████████| 19320/19320 [13:32<00:00, 23.77it/s] 


                                                text  \
0  <s> well everyone got up and going this mornin...   
1  <s> its still raining but thats okay with me. ...   
2                    <s> sort of suits my mood. </s>   
3  <s> i could easily have stayed home in bed wit...   
4       <s> this has been a lot of rain though. </s>   

                                              source  length  
0  ./raw_data/blogs\1000331.female.37.indUnk.Leo.xml      10  
1  ./raw_data/blogs\1000331.female.37.indUnk.Leo.xml      10  
2  ./raw_data/blogs\1000331.female.37.indUnk.Leo.xml       7  
3  ./raw_data/blogs\1000331.female.37.indUnk.Leo.xml      16  
4  ./raw_data/blogs\1000331.female.37.indUnk.Leo.xml      10  


In [12]:
# Divide el DataFrame en conjuntos de entrenamiento (80%) y prueba (20%) de manera aleatoria
df_bac_train = df_bac.sample(frac=0.8, random_state=42)
df_bac_test = df_bac.drop(df_bac_train.index)

# Guarda los conjuntos de entrenamiento y prueba en archivos Parquet separados
df_bac_train.to_parquet('./data/train_test/BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_training.parquet', index=False)
df_bac_test.to_parquet('./data/train_test/BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_testing.parquet', index=False)

In [13]:
# Genera los modelos de N-gramas
unigram_counts_bac, unigram_bac = create_ngram_model(1, df_bac['text'])

In [14]:
bigram_counts_bac, bigram_bac = create_ngram_model(2, df_bac['text'])

In [15]:
trigram_counts_bac, trigram_bac = create_ngram_model(3, df_bac['text'])

In [16]:
# Guarda los modelos de N-gramas
save_ngram_model(unigram_counts_bac, unigram_bac, 'BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_unigrams')
save_ngram_model(bigram_counts_bac, bigram_bac, 'BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_bigrams')
save_ngram_model(trigram_counts_bac, trigram_bac, 'BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_trigrams')

BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_unigrams saved.
BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_bigrams saved.
BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_trigrams saved.
